In [1]:
import os
import ccxt
# import config
import schedule
import pandas as pd
pd.set_option('display.max_rows', None)
from datetime import datetime
import time
import pytz
import json
import numpy as np
# import warnings
# warnings.filterwarnings('ignore')
import sys
from importlib import reload  

In [2]:
# !. ./load-creds.sh

Error authenticating: error looking up token: Error making API request.

URL: GET http://127.0.0.1:8200/v1/auth/token/lookup-self
Code: 403. Errors:

* permission denied


In [2]:
def cancelOrders(exch, symbol):
  exchange = exchanges[exch]
  for order in exchange.fetch_open_orders(symbol):
    exchange.cancel_order(order['id'], symbol=order['symbol'])

In [3]:
def buyMarket__stoporderMarket(exchange, symbol, amount, triggerPrice, stopPrice):
  db.xqueue.insert_one({
    "description": "pktrade",
    "status": "new",
    "q": [
      {
        "qid": 0,
        "symbol": symbol,
        "status": "new",
        "trigger": f"ticker['ask'] < {triggerPrice}",
        "exchange": exchange,
        "method": f"exchange.create_market_buy_order('{symbol}', {amount})"
      },
      {
        "qid": 1,
        "symbol": symbol,
        "status": "new",
        "trigger": "'done' == qx['q'][0]['status']",
        "exchange": exchange,
        "method": f"exchange.create_order('{symbol}', 'STOP_LOSS_LIMIT', 'sell', {amount}, {stopPrice * 0.95}, {{'stopPrice': {stopPrice}}})"
      }
    ]
  })

In [4]:
def sellMarket__priceAbove(exchange, symbol, amount, triggerPrice):
  db.xqueue.insert_one({
    "description": "pktrade",
    "status": "new",
    "q": [
      {
        "qid": 0,
        "symbol": symbol,
        "status": "new",
        "trigger": f"ticker['bid'] >= {triggerPrice}",
        "exchange": exchange,
        "method": f"cancelOrders('{exchange}', '{symbol}')"
      },
      {
        "qid": 1,
        "symbol": symbol,
        "status": "new",
        "trigger": "'done' == qx['q'][0]['status']",
        "exchange": exchange,
        "method": f"exchange.create_order('{symbol}', 'market', 'sell', {amount})"
      }
    ]
  })

In [5]:
exchanges = {}
config = False
with open ('config.json') as js: 
        config = json.load(js)

for exkey, exconf in config['exchanges'].items():
        if exconf['enabled'] and not exkey in exchanges:
                excred = exconf['cred']
                for crk, crv in excred.items():
                        if crv[0] == "$" : 
                                excred[crk] = os.getenv(crv[1:])

                exchanges[exkey] = getattr(ccxt, exkey) (excred)

In [7]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

client = MongoClient()
db=client.pkbot

tickers = {}

iit = 0

def run_bot():
  # time.sleep(2)
  global exchanges, tickers, iit
  if (iit % 10) == 0: print(f"{datetime.now()} Scanning for new orders ({iit})...")
  iit += 1
  for csyms in db.xqueue.aggregate([    
      {'$match':{'status': "new"}},
      {'$unwind':"$q"},
      {'$group':{'_id': {'exchange':"$q.exchange", "symbol": "$q.symbol"}}},
      {'$group':{'_id': '$_id.exchange',"symbols": {'$addToSet': "$_id.symbol"}}},
      # {'$match':{'_id': qo['exchange']}}
  ]):
    exchange_name = csyms['_id']
    symbols = csyms['symbols']
    tickers[exchange_name] = exchanges[exchange_name].fetchTickers(symbols)
    # print(exchange_name, tickers[exchange_name])
  for qx in db.xqueue.find({'status' :'new'}):
    print(f"{datetime.now()} Processing q/{qx['_id']}")
    for idxqo, qo in enumerate(qx['q']):
      nqx = db.xqueue.find_one({'_id': qx.get('_id')})
      print(f'{datetime.now()}   qid/{qo["qid"]} status: {qo["status"]}')
      exchange_name = qo['exchange']
      symbol = qo['symbol']
      if qo['status'] != 'new':
        continue

      print(f'{datetime.now()}        Checking {exchange_name}: qid/{qo["qid"]} {qo["trigger"]}')
      
      exchange = exchanges[exchange_name]
      ticker = tickers[exchange_name][symbol]

      if not eval(qo['trigger'], {'ticker': ticker, 'qx' : nqx}):
        continue

      print(f'{datetime.now()}        Triggered {exchange_name}:  {qo["method"]}')
      # res = getattr(exchange, qo['method'])(*qo['margs'])
      res = eval(qo['method'])
      # print('RES:', res)
      updres = db.xqueue.update_one({'_id' : qx.get('_id') }, {
        '$set': { 
          # f"q.{idxqo}.orderId": res['id'],
          f"q.{idxqo}.status": 'done',
          f"q.{idxqo}.res": res,
          }
      })

    if db.xqueue.find_one({'_id': qx.get('_id'), "q.status" : { "$ne": "done" }}) == None:
      db.xqueue.update_one({'_id' : qx.get('_id') }, {
          '$set': { 'status': 'done' }
        })

schedule.every(2).seconds.do(run_bot)
while True:
    schedule.run_pending()
    time.sleep(1)
# run_bot()

2021-09-24 15:44:18.275043 Scanning for new orders (0)...
2021-09-24 15:44:38.344871 Scanning for new orders (10)...
2021-09-24 15:44:58.409491 Scanning for new orders (20)...
2021-09-24 15:45:18.488822 Scanning for new orders (30)...


KeyboardInterrupt: 